# WindRose Plot

In [682]:
import pandas as pd
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap
import seaborn as sns
import datetime as dt
from bisect import bisect_left
import math
%matplotlib inline
import colorlover as cl
import numpy as np
import plotly
import plotly.io as pio
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
print (__version__) # requires version >= 1.9.0
plotly.tools.set_credentials_file(username='fmmaciel', api_key='6Tuz22MRwaqXLIHcB77V')
init_notebook_mode(connected=True)

def takeClosest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.
    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
       return after
    else:
       return before

def deg2Compass(num):
    val=int((num/22.5)+.5)
    arr=["N","NNE","NE","ENE","E","ESE", "SE", "SSE","S","SSO","SO","OSO","O","ONO","NO","NNO"]
    return arr[(val % 16)]

def windvelBins(vel):
    limits = [0.5, 1, 2, 3, 4, 5, 6, 7]
    bins = ['calm', '0,5-1','1-2','2-3','3-4','4-5','5-6','6-7','7+']
    for i in range(0,len(limits)):
        if vel < limits[i]:
            return bins[i]
    return bins[-1]

3.4.1


In [683]:
directory = str('D:/Mestrado/1.Resultados/Meteorologia/')
output = str('C:/Users/50033096/OneDrive - ArcelorMittal/Mestrado UFMG/1. DISSERTAÇÃO/ACT/')
mes   = ['jan','jul']
data  = ['01-09','07-24']
periodo = ['Verão', 'Inverno']
grade = 'd04'
#modelagens = ['andrade2015','rizzieri2016','rizzieri2016_nudge','silvajunior2009']
modelagens = ['andrade2015','rizzieri2016','rizzieri2016_nudge']
mod_var = ['T2']
est_var = ['temp','ur', 'wind_vel','wind_dir', 'prec']
spinup = 48
gmt = 3
date_index = 0

In [684]:
#DADOS ESTACAO
estacao     = 'SP_A726_PIRACICABA'
estcidade   = 'PIRACICABA'
estlat      = -22.703132
estlon      = -47.623317
altitude    = 566
est = pd.read_csv(directory+'arqEstacao/'+estacao+'.csv', delimiter=';')
est['datetime'] = (est['year'].map(str)+'-'+
                  est['month'].map(str)+'-'+
                  est['day'].map(str)+'-'+
                  est['hour'].map(str))
est['datetime'] = est['datetime'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d-%H'))
est.set_index('datetime',inplace=True)

#DADOS RODADAS
rodada=[]
for mod in modelagens:
    rodada.append(Dataset(directory+'arqWRF/'+mes[date_index]+
                        '/wrf_fnl_pira_'+mod+
                        '_'+mes[date_index]+
                        '/wrfout_'+grade+
                        '_2015-'+data[date_index]+'_00%3A00%3A00'))

#CROP DE DATA E COORDENADAS PARA POSICIONAR ESTAÇÃO
time_i   = dt.datetime.strptime(rodada[0].START_DATE,'%Y-%m-%d_%H:%M:%S')
lats     = rodada[0].variables['XLAT'][0,:,:]
longs    = rodada[0].variables['XLONG'][0,:,:]
if (estlat > np.amin(lats) and
    estlat < np.amax(lats) and
    estlon > np.amin(longs)and
    estlon < np.amax(longs)):                
    esty     = lats[:,0].tolist().index(takeClosest(lats[:,0],estlat))     #index Lat da estação no domínio
    estx     = longs[0,:].tolist().index(takeClosest(longs[0,:],estlon))

In [685]:
# Iniciar DF com dados da Estação, usando est_param
windveldf = pd.DataFrame(est[time_i:time_i+dt.timedelta(hours=rodada[0].dimensions['Time'].size-1)]['wind_vel'])
winddirdf = pd.DataFrame(est[time_i:time_i+dt.timedelta(hours=rodada[0].dimensions['Time'].size-1)]['wind_dir'])
urdf['press']      = pd.DataFrame(est[time_i:time_i+dt.timedelta(hours=rodada[0].dimensions['Time'].size-1)]['press'])

# Tratar valores 0 
windveldf = windveldf.applymap(lambda x: np.nan if x == -999 else x)
winddirdf = winddirdf.applymap(lambda x: np.nan if x == -999 else x)

# Converter dados da estação em -180 a 180
winddirdf = winddirdf.applymap(lambda x: x - 360 if x > 180 else x)

# MODELAGENS 
# Buscar dados das modelagens, usando mod_param
for i in range(0,len(modelagens)):
# VENTOS   
    u10 = 'U10'+modelagens[i]
    v10 = 'V10'+modelagens[i]
    # Calcula Velocidade
    windveldf[u10] = pd.Series(rodada[i].variables['U10'][:,esty,estx], index=windveldf.index)
    windveldf[v10] = pd.Series(rodada[i].variables['V10'][:,esty,estx], index=windveldf.index)
    windveldf[modelagens[i]] = ((windveldf[u10]**2 + windveldf[v10]**2)**0.5)
    windveldf.drop(columns=[u10,v10], inplace=True)
    # Calcula Direção
    winddirdf[u10] = pd.Series(rodada[i].variables['U10'][:,esty,estx], index=winddirdf.index)
    winddirdf[v10] = pd.Series(rodada[i].variables['V10'][:,esty,estx], index=winddirdf.index) 
    direc = []
    for row in range(0,len(winddirdf)):
        if winddirdf[u10].iloc[row] > 0:
            d = 90 - ((180.0/np.pi) * math.atan(winddirdf[v10].iloc[row]/winddirdf[u10].iloc[row])) + 180
            if d > 180:
                d = d - 360
                direc.append(d)
            else:
                direc.append(d)
        else:
            d = 90 - ((180.0/np.pi) * math.atan(winddirdf[v10].iloc[row]/winddirdf[u10].iloc[row]))
            if d > 180:
                d = d - 360
                direc.append(d)
            else:
                direc.append(d)
    winddirdf[modelagens[i]] = pd.Series(direc, index=winddirdf.index)
    winddirdf.drop(columns=[u10,v10], inplace=True)
      
# RENOMEAR COLUNAS DA ESTACAO
windveldf.rename(columns={'wind_vel': estacao}, inplace=True)
winddirdf.rename(columns={'wind_dir': estacao}, inplace=True)

In [671]:
windveldf.drop(windveldf.index[:spinup] | windveldf.index[spinup+24*7:], inplace=True)
winddirdf.drop(winddirdf.index[:spinup] | winddirdf.index[spinup+24*7:], inplace=True)

In [672]:
windveldf.index -= pd.Timedelta(gmt,'h')
winddirdf.index -= pd.Timedelta(gmt,'h')

In [673]:
# Velocidade do Vento
stackdf = windveldf.stack()
stackdf = pd.DataFrame(stackdf.reset_index())
stackdf.columns = ['dt','rodadas','windvel']

stackdir = winddirdf.stack()
stackdir = pd.DataFrame(stackdir.reset_index())
stackdir.columns = ['dt','rodadas','winddir']
stackdf['winddir'] = stackdir['winddir']
stackdf['winddir'] = stackdf['winddir'].apply(lambda x: x+360 if x<0 else x)

In [688]:
rod = stackdf.rodadas.unique()
roduse = rod[3]

In [689]:
plotwind = stackdf[stackdf['rodadas']==roduse]
plotwind = plotwind.copy()
plotwind['r'] = plotwind['winddir'].apply(deg2Compass)
plotwind['windvelBINS'] = plotwind['windvel'].apply(windvelBins)
plotwind.drop(columns=['dt', 'rodadas', 'winddir','windvel'], inplace=True)
plotwind['percent'] = np.ones(len(plotwind['windvelBINS']))/len(plotwind['windvelBINS'])
plotwind = plotwind.pivot_table(index='r',columns='windvelBINS',aggfunc=sum)
plotwind.fillna(value=0.0, inplace=True)
cols=plotwind.columns.tolist()
cols = cols[-1:] + cols[:-1]
plotwind=plotwind[cols]
plotwind.columns = plotwind.columns.droplevel()
plotwind = plotwind.applymap(lambda x: x*100)
plotwind.reset_index(inplace=True)
sorter=["N","NNE","NE","ENE","E","ESE", "SE", "SSE","S","SSO","SO","OSO","O","ONO","NO","NNO"]
sorterIndex = dict(zip(sorter,range(len(sorter))))
plotwind['r_Rank'] = plotwind['r'].map(sorterIndex)
plotwind.sort_values(by='r_Rank', inplace=True)
plotwind.drop(columns='r_Rank', inplace=True)

In [676]:
plotwind

windvelBINS,r,calm,"0,5-1",1-2,2-3,3-4,4-5,5-6
3,N,1.190476,0.595238,6.547619,1.190476,0.000000,0.000000,0.000000
5,NNE,0.000000,0.595238,2.380952,0.595238,0.000000,0.000000,0.000000
4,NE,0.595238,1.190476,1.785714,0.000000,0.595238,0.000000,0.000000
1,ENE,0.000000,1.785714,10.119048,1.785714,0.000000,0.000000,0.000000
0,E,0.595238,1.190476,8.333333,4.761905,1.190476,0.000000,0.000000
2,ESE,0.595238,1.190476,0.595238,0.595238,1.785714,0.595238,0.000000
12,SE,0.000000,1.785714,0.000000,0.000000,0.595238,0.595238,1.785714
14,SSE,0.000000,0.595238,0.000000,0.595238,0.595238,0.000000,0.595238
11,S,0.000000,0.595238,0.595238,0.000000,0.000000,0.000000,0.595238
15,SSO,0.000000,0.595238,5.357143,2.380952,0.000000,0.000000,0.000000


In [692]:
dat = []
counter = 0
colorsnum = 9
cls = ['BuPu','PuBu','RdPu','YlOrBr']
for col in plotwind.columns:
    if col != 'r':
        if col == 'calm':
            n = col
        else:
            n = col+' m/s'
        dat.append(
            go.Area(t=plotwind['r'],
                    r=plotwind[col],
                    marker=dict(color=cl.scales[str(colorsnum)]['seq'][cls[1]][counter]),
                    name=n ) )
        counter+=1

fig = go.Figure(data=dat, layout=go.Layout(orientation=270, barmode='stack', radialaxis=dict(ticksuffix='%', range=[0,20])))

In [693]:
py.iplot(fig)

In [694]:
pio.write_image(fig, output+'RosePlot_'+roduse+mes[date_index]+'.png')

In [680]:
mes[date_index]

'jan'

In [681]:
roduse

'rizzieri2016_nudge'

Index(['r', '0-1', '1-2', '2-3', '3-4', '4-4', '4-5', '5-6', '6+'], dtype='object')

,r,0-1,1-2,2-3,3-4,4-4,4-5,5-6,6+
0,N,0.5,1.6,0.9,0.9,0.4,0.30,0.20,0.10
1,NNE,0.6,1.8,1.3,0.8,0.5,0.30,0.10,0.10
2,NE,0.5,1.5,1.6,1.2,1.2,0.60,0.10,0.10
3,ENE,0.4,1.6,0.9,1.0,0.5,0.20,0.10,0.10
4,E,0.4,1.6,1.0,0.8,0.4,0.10,0.10,0.10
5,ESE,0.3,1.2,0.6,0.4,0.2,0.10,0.10,0.05
6,SE,0.4,1.5,0.6,0.5,0.4,0.05,0.05,0.05
7,SSE,0.4,1.7,0.9,0.5,0.4,0.10,0.05,0.05
8,S,0.6,2.2,1.4,0.8,0.7,0.10,0.10,0.05
9,SSW,0.4,2.0,1.7,0.9,0.6,0.20,0.05,0.10


In [67]:
dat = []
counter = 0
for col in df.columns:
    if col != 'r':
        dat.append(
            go.Area(t=df['r'],
                    r=df[col],
                    marker=dict(color=cl.scales['9']['seq']['PuBu'][counter]),
                    name=col+' m/s' ) )
        counter+=1

fig = go.Figure(data=dat, layout=go.Layout(orientation=270, barmode='stack'))

In [40]:
py.iplot(fig)
pio.write_image(fig, 'images/fig1.png')

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/wind_rose.csv')
df.columns